In [30]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from google.colab import drive
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')
def load_embeddings(file_path):
    data = torch.load(file_path, map_location=torch.device('cpu'))
    embeddings = data['embeddings']  # Shape: (num_samples, embedding_dim)
    labels = data['labels']          # Shape: (num_samples,)
    return embeddings, labels

file_path1 = "/content/drive/MyDrive/fine_tuned_train_embeddings.pt"
file_path2 = "/content/drive/MyDrive/fine_tuned_val_embeddings.pt"
file_path3 = "/content/drive/MyDrive/fine_tuned_test_embeddings.pt"
train_embeddings, train_labels = load_embeddings(file_path1)
val_embeddings, val_labels = load_embeddings(file_path2)
test_embeddings, test_labels = load_embeddings(file_path3)
train_Y= torch.tensor(train_labels.values) if isinstance(train_labels, pd.Series) else torch.tensor(train_labels)
val_Y = torch.tensor(val_labels.values) if isinstance(val_labels, pd.Series) else torch.tensor(val_labels)
test_Y = torch.tensor(test_labels.values) if isinstance(test_labels, pd.Series) else torch.tensor(test_labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-30-24d60ef19bbb>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path, map_location=torch.device('cpu'))


In [23]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
file_path2 = '/content/drive/MyDrive/finalDataSet.csv'
df = pd.read_csv(file_path2)
Y = df['MisconceptionId'].astype(int)
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
print(Y[:5])
Y_tensor = torch.tensor(Y.values) if isinstance(Y, pd.Series) else torch.tensor(Y)


[1347  792  724  425  207]
tensor([1347,  792,  724,  425,  207])


In [31]:
import torch.nn as nn

# Define a simple linear classifier model
class LinearClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearClassifier, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# Get the input size from embeddings and the number of classes from Y
input_dim = test_embeddings.shape[1]  # Embedding dimension
max_label_value = Y_tensor.max().item()

# Initialize the model
model = LinearClassifier(input_dim=input_dim, output_dim=max_label_value + 1)


In [32]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

learning_rate = 1e-5
batch_size = 32
num_epochs = 250

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0.0
    model.train()  # Set the model to training mode

    # Shuffle training data at the beginning of each epoch
    perm = torch.randperm(train_embeddings.shape[0])
    train_embeddings_shuffled = train_embeddings[perm]
    train_Y_shuffled = train_Y[perm]

    # Process inputs in batches for training
    for i in range(0, train_embeddings.shape[0], batch_size):
        batch_embeddings = train_embeddings_shuffled[i:i + batch_size]
        batch_labels = train_Y_shuffled[i:i + batch_size]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_embeddings)

        # Compute loss
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        epoch_loss += loss.item()

    # Validation every 10 epochs
    if (epoch + 1) % 25 == 0:
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            val_outputs = model(val_embeddings)
            _, val_preds = torch.max(val_outputs, dim=1)
            val_accuracy = accuracy_score(val_Y.cpu(), val_preds.cpu())  # Calculate accuracy
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch [25/250], Loss: 539.5733, Validation Accuracy: 0.2174
Epoch [50/250], Loss: 343.5833, Validation Accuracy: 0.2426
Epoch [75/250], Loss: 227.7819, Validation Accuracy: 0.2654
Epoch [100/250], Loss: 160.5357, Validation Accuracy: 0.2654
Epoch [125/250], Loss: 120.9631, Validation Accuracy: 0.2609
Epoch [150/250], Loss: 96.9407, Validation Accuracy: 0.2677
Epoch [175/250], Loss: 82.4920, Validation Accuracy: 0.2700
Epoch [200/250], Loss: 73.3122, Validation Accuracy: 0.2677
Epoch [225/250], Loss: 67.0864, Validation Accuracy: 0.2632
Epoch [250/250], Loss: 61.5620, Validation Accuracy: 0.2654


In [33]:

# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(test_embeddings)
    _, test_preds = torch.max(test_outputs, dim=1)
    test_accuracy = accuracy_score(test_Y.cpu(), test_preds.cpu())  # Calculate accuracy
print(f"Test Accuracy: {test_accuracy:.8f}")

Test Accuracy: 0.26544622


In [34]:
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

# After making predictions for validation or test set
test_outputs2 = model(test_embeddings)
_, test_preds2 = torch.max(test_outputs, dim=1)

# Use sklearn to calculate classification report
report = classification_report(test_Y.cpu(), test_preds2.cpu(), output_dict=True, zero_division=0)


precision, recall, f1, _ = precision_recall_fscore_support(test_Y.cpu(), test_preds2.cpu(), average='macro', zero_division=0)

print(f"Test Macro-Precision: {precision:.4f}")
print(f"Test Macro-Recall: {recall:.4f}")
print(f"Test Macro-F1: {f1:.4f}")
# Extract macro-accuracy from the classification report
macro_accuracy = report['accuracy']  # sklearn's accuracy report is macro-averaged for multi-class

print(f"Test Macro-Accuracy: {macro_accuracy:.8f}")

Test Macro-Precision: 0.1605
Test Macro-Recall: 0.1615
Test Macro-F1: 0.1560
Test Macro-Accuracy: 0.26544622


In [35]:
import numpy as np
import torch

def mean_average_precision_at_k(predictions, actuals, k=25):
    """
    Calculate the MAP@k for a set of predictions and actual labels.

    Parameters:
    - predictions: List of lists containing predicted labels, ranked from highest to lowest confidence.
    - actuals: List of correct labels corresponding to the predictions.
    - k: The cutoff rank for calculating MAP (default is 25).

    Returns:
    - The MAP@k score.
    """
    average_precisions = []

    for pred, actual in zip(predictions, actuals):
        # We stop once we find the first correct label
        precision_at_k = 0.0  # Initialize precision
        for i in range(min(len(pred), k)):  # Loop up to k
            if pred[i] == actual:
                precision_at_k = 1 / (i + 1)  # Precision at rank i+1
                break  # Stop once the correct label is found

        average_precisions.append(precision_at_k)  # Append precision for this observation

    # Calculate and return the mean average precision across all observations
    return np.mean(average_precisions)


# Modify your model evaluation code to return top 25 predictions
def evaluate_model_map_at_25(model, test_embeddings, test_Y, k=25):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        # Get model outputs (logits) for test data
        test_outputs = model(test_embeddings)

        # Get the top 25 predictions for each test sample
        topk_probs, topk_indices = torch.topk(test_outputs, k=k, dim=1)  # Get top-25 predictions

        # Convert the predictions to numpy for processing
        topk_preds = topk_indices.cpu().numpy()  # Shape: (num_samples, 25)
        test_Y_np = test_Y.cpu().numpy()         # Ground truth labels

        # Calculate MAP@25 using the mean_average_precision_at_k function
        map_at_25 = mean_average_precision_at_k(topk_preds, test_Y_np, k=k)

    print(f"Test MAP@25: {map_at_25:.4f}")
    return map_at_25

# Call this function during the test evaluation phase
map_at_25 = evaluate_model_map_at_25(model, test_embeddings, test_Y, k=25)


Test MAP@25: 0.4009
